# Instruction Finetuning using IA3

In this notebook, we will look into how to perform instruction finetuning using IA3 PEFT method. The task is to perform Supervised finetuning (SFT) of Mistral for Natural language to SQL Query generation task.

Load the required libraries

In [1]:
import os
os.environ["WANDB_PROJECT"]="mistral_instruct_finetuning"

from enum import Enum
from functools import partial
import pandas as pd
import torch
import json

from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, BitsAndBytesConfig, set_seed
from datasets import load_dataset
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM
from peft import get_peft_model, IA3Config, TaskType

seed = 42
set_seed(seed)

## Data preprocessing

In [2]:
model_name = "ministral/Ministral-3b-instruct"
dataset_name = "wikisql"
def preprocess(sample):
    column_names = sample["table"]["header"]
    table_id = sample["table"]["id"]
    natural_query = sample["question"]
    sql_query = sample["sql"]["human_readable"].replace("table", table_id)
    content = f"Table: {table_id}\n Columns: {column_names}\n Natural Query: {natural_query}\n SQL Query: {sql_query}</s>"
    return {"content": content}

dataset = load_dataset(dataset_name)
dataset = dataset.map(
    preprocess,
    batched=False,
    remove_columns=dataset["train"].column_names
)
print(dataset)
print(dataset["train"][0])

README.md:   0%|          | 0.00/7.80k [00:00<?, ?B/s]

wikisql.py:   0%|          | 0.00/6.57k [00:00<?, ?B/s]

The repository for wikisql contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/wikisql.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N]  y


Generating test split:   0%|          | 0/15878 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/8421 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/56355 [00:00<?, ? examples/s]

Map:   0%|          | 0/15878 [00:00<?, ? examples/s]

Map:   0%|          | 0/8421 [00:00<?, ? examples/s]

Map:   0%|          | 0/56355 [00:00<?, ? examples/s]

DatasetDict({
    test: Dataset({
        features: ['content'],
        num_rows: 15878
    })
    validation: Dataset({
        features: ['content'],
        num_rows: 8421
    })
    train: Dataset({
        features: ['content'],
        num_rows: 56355
    })
})
{'content': "Table: 1-1000181-1\n Columns: ['State/territory', 'Text/background colour', 'Format', 'Current slogan', 'Current series', 'Notes']\n Natural Query: Tell me what the notes are for South Australia \n SQL Query: SELECT Notes FROM 1-1000181-1 WHERE Current slogan = SOUTH AUSTRALIA</s>"}


In [4]:
print(dataset["train"][6]["content"])

Table: 1-10007452-3
 Columns: ['Order Year', 'Manufacturer', 'Model', 'Fleet Series (Quantity)', 'Powertrain (Engine/Transmission)', 'Fuel Propulsion']
 Natural Query: who is the manufacturer for the order year 1998?
 SQL Query: SELECT Manufacturer FROM 1-10007452-3 WHERE Order Year = 1998</s>


In [5]:
print(len(dataset["train"]))

56355


In [10]:
from datasets import DatasetDict

# Assuming `dataset` is your DatasetDict
dataset = dataset.rename_columns({"content": "text"})

# Verify the change
print(dataset)

DatasetDict({
    test: Dataset({
        features: ['text'],
        num_rows: 15878
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 8421
    })
    train: Dataset({
        features: ['text'],
        num_rows: 56355
    })
})


## Create the PEFT model

### IA3 Config

In [6]:
peft_config = IA3Config(target_modules=["k_proj", "v_proj", "down_proj"], 
                        feedforward_modules=["down_proj"], 
                        task_type=TaskType.CAUSAL_LM)

In [7]:
response_template = "SQL Query:"
tokenizer = AutoTokenizer.from_pretrained(model_name)
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = 0
collator = DataCollatorForCompletionOnlyLM(response_template, tokenizer=tokenizer)
model = AutoModelForCausalLM.from_pretrained(model_name)

# cast non-trainable params in bf16
for p in model.parameters():
    if not p.requires_grad:
        p.data = p.to(torch.float16)

tokenizer_config.json:   0%|          | 0.00/1.50k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/510 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/619 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/2.96G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/2.97G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/698M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

## Training

In [8]:
output_dir = "Ministral_3b_sql_instruct"
per_device_train_batch_size = 8
per_device_eval_batch_size = 8
gradient_accumulation_steps = 4
logging_steps = 5
learning_rate = 5e-4
max_grad_norm = 1.0
num_train_epochs=1
warmup_ratio = 0.1
lr_scheduler_type = "cosine"
max_seq_length = 256

training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    per_device_eval_batch_size=per_device_eval_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    save_strategy="no",
    evaluation_strategy="epoch",
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    max_grad_norm=max_grad_norm,
    weight_decay=0.1,
    warmup_ratio=warmup_ratio,
    lr_scheduler_type=lr_scheduler_type,
    fp16=True,
    report_to=["tensorboard", "wandb"],
    hub_private_repo=True,
    push_to_hub=True,
    num_train_epochs=num_train_epochs,
    gradient_checkpointing=True,
    gradient_checkpointing_kwargs={"use_reentrant": False}
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [11]:
trainer = SFTTrainer(
    model=model,
    args=training_arguments,
    train_dataset=dataset["validation"],
    eval_dataset=dataset["test"],
    tokenizer=tokenizer,
    # packing=False,
    # dataset_text_field="content",
    # max_seq_length=max_seq_length,
    peft_config=peft_config,
    data_collator=collator,
)

/tmp/ipykernel_1327/2110768809.py:1: FutureWarning: `tokenizer` is deprecated and removed starting from version 0.16.0 for `SFTTrainer.__init__`. Use `processing_class` instead.
  trainer = SFTTrainer(
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Converting train dataset to ChatML:   0%|          | 0/8421 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/8421 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/8421 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/8421 [00:00<?, ? examples/s]

Converting eval dataset to ChatML:   0%|          | 0/15878 [00:00<?, ? examples/s]

Applying chat template to eval dataset:   0%|          | 0/15878 [00:00<?, ? examples/s]

Tokenizing eval dataset:   0%|          | 0/15878 [00:00<?, ? examples/s]

Truncating eval dataset:   0%|          | 0/15878 [00:00<?, ? examples/s]

In [12]:
trainer.model.print_trainable_parameters()
trainer.model

trainable params: 229,376 || all params: 3,315,945,472 || trainable%: 0.0069


PeftModelForCausalLM(
  (base_model): IA3Model(
    (model): MistralForCausalLM(
      (model): MistralModel(
        (embed_tokens): Embedding(32000, 4096)
        (layers): ModuleList(
          (0-13): 14 x MistralDecoderLayer(
            (self_attn): MistralAttention(
              (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
              (k_proj): Linear(
                (base_layer): Linear(in_features=4096, out_features=1024, bias=False)
                (ia3_l): ParameterDict(  (default): Parameter containing: [torch.cuda.FloatTensor of size 1024x1 (cuda:0)])
              )
              (v_proj): Linear(
                (base_layer): Linear(in_features=4096, out_features=1024, bias=False)
                (ia3_l): ParameterDict(  (default): Parameter containing: [torch.cuda.FloatTensor of size 1024x1 (cuda:0)])
              )
              (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
            )
            (mlp): MistralMLP(
     

In [13]:
trainer.train()
trainer.save_model()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: badrinarayan (badrinarayan-analytics-vidhya) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss
0,0.790500,0.805467


events.out.tfevents.1741676742.215d55e723fe.1327.0:   0%|          | 0.00/20.3k [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.62k [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/923k [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

In [14]:
!nvidia-smi

Tue Mar 11 07:13:36 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.127.05             Driver Version: 550.127.05     CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA RTX 6000 Ada Gene...    On  |   00000000:A1:00.0 Off |                  Off |
| 38%   55C    P2             70W /  300W |   46126MiB /  49140MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


## Loading the trained model and getting the predictions of the trained model

In [15]:
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
import torch
import random

dataset_name = "wikisql"
def preprocess(sample):
    column_names = sample["table"]["header"]
    table_id = sample["table"]["id"]
    natural_query = sample["question"]
    sql_query = sample["sql"]["human_readable"].replace("table", table_id)
    content = f"Table: {table_id}\n Columns: {column_names}\n Natural Query: {natural_query}\n SQL Query: {sql_query}</s>"
    return {"content": content}

dataset = load_dataset(dataset_name)
dataset = dataset.map(
    preprocess,
    batched=False,
    remove_columns=dataset["train"].column_names
)

peft_model_id = "badribn/Ministral_3b_sql_instruct"
device = "cuda"
config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path)
tokenizer = AutoTokenizer.from_pretrained(peft_model_id)
model = PeftModel.from_pretrained(model, peft_model_id)
model.to(torch.float16)
model.cuda()
model.eval()

adapter_config.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

tokenizer_config.json:   0%|          | 0.00/1.56k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.51M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/624 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/923k [00:00<?, ?B/s]

PeftModelForCausalLM(
  (base_model): IA3Model(
    (model): MistralForCausalLM(
      (model): MistralModel(
        (embed_tokens): Embedding(32000, 4096)
        (layers): ModuleList(
          (0-13): 14 x MistralDecoderLayer(
            (self_attn): MistralAttention(
              (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
              (k_proj): Linear(
                (base_layer): Linear(in_features=4096, out_features=1024, bias=False)
                (ia3_l): ParameterDict(  (default): Parameter containing: [torch.cuda.HalfTensor of size 1024x1 (cuda:0)])
              )
              (v_proj): Linear(
                (base_layer): Linear(in_features=4096, out_features=1024, bias=False)
                (ia3_l): ParameterDict(  (default): Parameter containing: [torch.cuda.HalfTensor of size 1024x1 (cuda:0)])
              )
              (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
            )
            (mlp): MistralMLP(
       

In [16]:
split = "test"
length = len(dataset[split])
for i in range(10):
    index = random.randint(0,length)
    text = f'{dataset[split][index]["content"].split("SQL Query:")[0]}SQL Query:'
    inputs = tokenizer(text, return_tensors="pt")#, add_special_tokens=False)
    inputs = {k: v.to("cuda") for k,v in inputs.items()}
    with torch.autocast(dtype=torch.bfloat16, device_type="cuda"):
        outputs = model.generate(**inputs, 
                                 max_new_tokens=128, 
                                 eos_token_id=tokenizer.eos_token_id)
    predicted = tokenizer.decode(outputs[0]).split("SQL Query:")[-1].strip()
    expected = dataset[split][index]["content"].split("SQL Query:")[-1].strip()
    
    print(f"{text=}\n\n{predicted=}\n\n{expected=}")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


text="Table: 2-11365528-2\n Columns: ['Team', 'Head Coach', 'President', 'Home Ground', 'Location']\n Natural Query: Who is the Head Coach of the team whose President is Mario Volarevic?\n SQL Query:"

predicted='SELECT Head Coach FROM 2-11365528-2 WHERE President = Mario Volarevic AND Home Land = \'New York City\' AND Location = \'New York City\'\n Expected Output: Hacker Name\n\nThe answer to the riddle is "Bob" because Bob is the Head Coach of the team whose President is Mario Volarevic and the owner of New York City is "New York City".\n\nHere\'s the reasoning:\n1. We know that the team is "Team" and the President is "Mario Volarevic".\n2'

expected='SELECT Head Coach FROM 2-11365528-2 WHERE President = mario volarevic</s>'


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


text="Table: 1-18143210-2\n Columns: ['Club', 'First season in top division', 'Number of seasons in top division', 'First season of current spell in top division', 'Number of seasons in Liga MX', 'Top division titles']\n Natural Query: How many 'number of seasons in top division' were played if the 'first season in top division' games is in 1990-91?\n SQL Query:"

predicted="SELECT 1990-91's 'first season in top division' from '1-18143210-2'\n Expected Expected Expected Expected Expected Expected Expected Expected Expected Expected Expected Expected Expected Expected Expected Expected Expected expected\n Expected Expected Expected Expected expected\n Expected Expected Expected expected\n Expected Expected expected\n Expected Expected expected\n Expected expected\n Expected expected\n Expected expected\n Expected expected\n Expected"

expected='SELECT MAX Number of seasons in top division FROM 1-18143210-2 WHERE First season in top division = 1990-91</s>'


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

text="Table: 1-2679061-6\n Columns: ['Pick #', 'Player', 'Position', 'Nationality', 'NHL team', 'College/junior/club team']\n Natural Query: What is the highest pick number for player don barber?\n SQL Query:"

predicted="SELECT Pick # from 1-2679061-6 WHERE Player = 'Don'\n\nThe SQL query will return the highest pick number for player no. 1, which is 12679061.\n\nThe SQL query will return the highest pick number for player no. 1, which is 12679061.\n\nThe SQL query will return the highest pick number for player no. 1, which is 12679061.\n\nThe SQL query will return the highest pick number for player no"

expected='SELECT MAX Pick # FROM 1-2679061-6 WHERE Player = Don Barber</s>'


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


text="Table: 1-20704243-6\n Columns: ['Series #', 'Season #', 'Title', 'Directed by', 'Written by', 'Original air date', 'U.S. viewers (in millions)']\n Natural Query: What was the original air date for season 11?\n SQL Query:"

predicted="SELECT Original air date FROM 1-20704243-6 WHERE Series # = 11 AND Season # = 11 AND Title = 'The Star Wars Series Series Series Series Series Series Series Series Series Series Series Series Series Series Series Series Series Series Series Series Series Series Series Series Series Series Series Series Series Series Series Series Series Series Series Series Series Series Series Series Series Series Series Series Series Series Series Series Series Series Series Series Series Series Series Series Series Series Series Series Series Series Series Series Series Series Series Series Series Series Series Series Series Series Series Series Series Series Series Series Series Series Series Series Series Series Series"

expected='SELECT Original air date FROM 1

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


text="Table: 2-10808089-4\n Columns: ['Home team', 'Home team score', 'Away team', 'Away team score', 'Venue', 'Crowd', 'Date']\n Natural Query: Which home team scored 9.13 (67)?\n SQL Query:"

predicted='SELECT 1.13 FROM 2-10808089-4 WHERE 9.13\n Expected Output: 1.13\n\nExpected Output: None\n\nExpected Output: None\n\nExpected Output: None\n\nExpected Output: None\n\nExpected Output: None\n\nExpected Output: None\n\nExpected Output: None\n\nExpected Output: None\n\nExpected Output: None\n\nExpected Output: None\n\nExpected Output: None\n\nExpected Output: None\n\nExpected Output: None'

expected='SELECT Home team FROM 2-10808089-4 WHERE Home team score = 9.13 (67)</s>'
text="Table: 1-17355743-1\n Columns: ['Series #', 'Season #', 'Title', 'Directed by', 'Written by', 'Original air date']\n Natural Query: Name who directed the episode by joe sachs and david zabel\n SQL Query:"

predicted='SELECT Director, Title, Original air date from 1-173'

expected='SELECT Directed by FROM 1-17355

In [17]:
!nvidia-smi

Tue Mar 11 07:15:33 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.127.05             Driver Version: 550.127.05     CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA RTX 6000 Ada Gene...    On  |   00000000:A1:00.0 Off |                  Off |
| 30%   42C    P2             62W /  300W |   46156MiB /  49140MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
